In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_19368\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
base_url = "https://www.justwatch.com"
link = "https://www.justwatch.com/us/movies?genres=act&release_year_from=2022"

In [ ]:
titleLinks = []
soup=BeautifulSoup(page_source,'html.parser')
productlist = soup.find("div",{"listlayout":"Grid"})
titles = productlist.find_all("div", {"class":"title-list-grid__item"})
for title in titles:
    title_link = title.find("a").get('href')
    title_link = base_url+title_link
    titleLinks.append(title_link)
    
print(len(titleLinks))

In [5]:
# Python3 Program to Create list 
# with integers within given range 
  
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
      
genres = ['cmy', 'act', 'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
years = createList(1900, 2023)


search_links = []
exclude_link = "https://www.justwatch.com/us/movies?exclude_genres=act&exclude_genres=cmy&exclude_genres=doc&exclude_genres=fnt&exclude_genres=hrr&exclude_genres=msc&exclude_genres=rma&exclude_genres=spt&exclude_genres=wsn&exclude_genres=rly&exclude_genres=war&exclude_genres=scf&exclude_genres=trl&exclude_genres=fml&exclude_genres=hst&exclude_genres=drm&exclude_genres=crm&exclude_genres=ani"
search_links.append(exclude_link)
for i in genres:
    for j in years:
        link = f"https://www.justwatch.com/us/movies?genres={i}&release_year_from={j}&release_year_until={j}"
        search_links.append(link)
        
search_links

['https://www.justwatch.com/us/movies?exclude_genres=act&exclude_genres=cmy&exclude_genres=doc&exclude_genres=fnt&exclude_genres=hrr&exclude_genres=msc&exclude_genres=rma&exclude_genres=spt&exclude_genres=wsn&exclude_genres=rly&exclude_genres=war&exclude_genres=scf&exclude_genres=trl&exclude_genres=fml&exclude_genres=hst&exclude_genres=drm&exclude_genres=crm&exclude_genres=ani',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1900&release_year_until=1900',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1901&release_year_until=1901',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1902&release_year_until=1902',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1903&release_year_until=1903',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1904&release_year_until=1904',
 'https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1905&release_year_until=1905',
 'https://www.justwatch.com/us

In [9]:
test = ["https://www.justwatch.com/us/movies?genres=cmy&release_year_from=1924&release_year_until=1924"]

In [10]:
titleLinks = []
for link in test:
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    titleLinks.append(title_link)

C:\Users\user\AppData\Local\Temp\ipykernel_19368\804137475.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chromedriver = webdriver.Chrome(ChromeDriverManager().install())


In [11]:
len(titleLinks)

33

In [ ]:
search_links[:2]

In [ ]:
for link in search_links[:3]:
    chrome_options = Options() 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    with driver as browser:
        browser.get(link)
        SCROLL_PAUSE_TIME = 5
        soups = []
        # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        titleLinks = []
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
            title_link = title.find("a").get('href')
            title_link = base_url+title_link
            titleLinks.append(title_link)

print(len(titleLinks))
    

In [ ]:
import time

browser = webdriver.Firefox()
browser.get("https://en.wikipedia.org")
browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(3)
browser.close()

In [ ]:
url = link
driver.get(url)
ScrollNumber = 50
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    WebDriverWait(driver, 10)
    time.sleep(5)
page_source =driver.page_source

In [ ]:
titleLinks = []
soup=BeautifulSoup(page_source,'html.parser')
productlist = soup.find("div",{"listlayout":"Grid"})
titles = productlist.find_all("div", {"class":"title-list-grid__item"})
for title in titles:
    title_link = title.find("a").get('href')
    title_link = base_url+title_link
    titleLinks.append(title_link)

In [ ]:
len(titleLinks)

In [ ]:
labels  = []
values = []
url = "https://www.justwatch.com/us/movie/uncharted-2017"
def get_detail(soup_object):
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

In [ ]:
def get_cast(soup_object):
    cast = []
    actors = soup_object.find_all("div", {"class":"title-credits__actor"})
    for actor in actors:
        actor_name = actor.text.replace('\n',"")
        cast.append(actor_name)
    return cast

In [ ]:
class Pricing:
    def __init__(self, soup_object):
        self.soup_object = soup_object

    def get_stream_options(self):
        stream_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)


        return stream_list


    def get_rent_options(self):
        rent_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list

    def get_buy_options(self):
        buy_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list


In [ ]:
def get_synopsis(soup_object):
    driver.get(soup_object)       
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

get_rent_options(soup)

In [ ]:
from pydoc import synopsis


data= []
for title in titleLinks:
    from datetime import datetime
    f = requests.get(title,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    prices = Pricing(hun)
    
    try:
        detail = get_detail(hun)
    except:
        detail = None
        
    try:
        title_name = hun.find("div", {"class":"title-block"}).text.replace('\n',"")
    except:
        title_name = None
    
    try:
        streaming_options = prices.get_stream_options()
    except:
        streaming_options = None
        
    try:
        renting_options = prices.get_rent_options()
    except:
        renting_options = None
    
    try:
        synopsis = get_synopsis(title)
    except:
        synopsis = None
    
    try:
        buying_options = prices.get_buy_options()
    except:
        buying_options = None
        
    try:
        cast = get_cast(hun)
    except:
        cast = None
    
    title_url = title
    
    
    
      
    tv = {"title":title_name, "detail":detail, "synopsis":synopsis, "streaming on":streaming_options, "buy from":buying_options, "rent on":renting_options,"cast":cast,"tite url": title_url}
    
    data.append(tv)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(data)
df

In [ ]:
df.to_csv("2022_action.csv")